In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn import ensemble
from sklearn.model_selection import cross_val_score

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Applications/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [4]:
#Look at all categorical variables and see how many distinct counts there are

categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [5]:
#Drop numeric features from dates
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [6]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [8]:
y2015.shape

(421095, 103)

In [9]:
y2015 = pd.get_dummies(y2015)

In [10]:
y2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421095 entries, 0 to 421094
Columns: 237 entries, id to verification_status_joint_Verified
dtypes: float64(87), uint8(150)
memory usage: 339.7 MB


In [11]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status_Current', 1)
Y = y2015['loan_status_Current']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.99833773, 0.99270957, 0.99085728, 0.98940869, 0.87197511,
       0.98556128, 0.76936047, 0.98938469, 0.82158208, 0.89273077])

In [12]:
null_counts = y2015.isnull().sum()
null_counts = pd.DataFrame(null_counts[null_counts > 0])
null_counts = null_counts.reset_index()
null_counts.columns = ['var', 'count']

In [13]:
null_counts

,var,count
0,mths_since_last_delinq,203962
1,mths_since_last_record,346680
2,mths_since_last_major_derog,298366
3,annual_inc_joint,420584
4,dti_joint,420586
5,open_acc_6m,399723
6,open_il_6m,399723
7,open_il_12m,399723
8,open_il_24m,399723
9,mths_since_rcnt_il,400285


In [14]:
drop_variables = list(null_counts['var'])

In [15]:
drop_variables

['mths_since_last_delinq',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'bc_open_to_buy',
 'bc_util',
 'mo_sin_old_il_acct',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_rev_accts',
 'num_tl_120dpd_2m',
 'percent_bc_gt_75']

In [16]:
y2015 = y2015.drop(['mths_since_last_delinq',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'bc_open_to_buy',
 'bc_util',
 'mo_sin_old_il_acct',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_rev_accts',
 'num_tl_120dpd_2m',
 'percent_bc_gt_75'], axis = 1)

In [17]:
#corr_matrix = y2015.corr().abs()

In [18]:
print("Data Frame")
print(y2015)
print()

print("Correlation Matrix")
print(y2015.corr())
print()

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(y2015, 3))

Data Frame
                id   member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0       68009401.0  72868139.0    16000.0      16000.0          16000.0   
1       68354783.0  73244544.0     9600.0       9600.0           9600.0   
2       68466916.0  73356753.0    25000.0      25000.0          25000.0   
3       68466961.0  73356799.0    28000.0      28000.0          28000.0   
4       68495092.0  73384866.0     8650.0       8650.0           8650.0   
5       68506798.0  73396623.0    23000.0      23000.0          23000.0   
6       68566886.0  73456723.0    29900.0      29900.0          29900.0   
7       68577849.0  73467703.0    18000.0      18000.0          18000.0   
8       66310712.0  71035433.0    35000.0      35000.0          35000.0   
9       68476807.0  73366655.0    10400.0      10400.0          10400.0   
10      68341763.0  72928789.0    20000.0      20000.0          20000.0   
11      68356421.0  73246236.0    22400.0      22400.0          22400.0   
12      684265

                                                 id  member_id  loan_amnt  \
id                                         1.000000   0.997540  -0.008288   
member_id                                  0.997540   1.000000  -0.008620   
loan_amnt                                 -0.008288  -0.008620   1.000000   
funded_amnt                               -0.008288  -0.008620   1.000000   
funded_amnt_inv                           -0.008554  -0.008885   0.999994   
int_rate                                  -0.053402  -0.053447   0.140572   
installment                               -0.012919  -0.013274   0.941205   
annual_inc                                 0.015055   0.014958   0.305734   
dti                                        0.005692   0.005806   0.006103   
delinq_2yrs                                0.002300   0.002261  -0.010424   
inq_last_6mths                             0.000660   0.000715  -0.039040   
open_acc                                   0.006309   0.006111   0.188214   

loan_amnt              funded_amnt              1.0
term_ 36 months        term_ 60 months          1.0
initial_list_status_f  initial_list_status_w    1.0
dtype: float64


In [19]:
print("Top Absolute Correlations")
print(get_top_abs_correlations(y2015, 100))

Top Absolute Correlations
loan_amnt                      funded_amnt                      1.000000
term_ 36 months                term_ 60 months                  1.000000
initial_list_status_f          initial_list_status_w            1.000000
purpose_educational            title_Learning and training      1.000000
application_type_INDIVIDUAL    application_type_JOINT           1.000000
out_prncp                      out_prncp_inv                    0.999997
total_pymnt                    total_pymnt_inv                  0.999997
loan_amnt                      funded_amnt_inv                  0.999994
funded_amnt                    funded_amnt_inv                  0.999994
open_acc                       num_sats                         0.998602
purpose_moving                 title_Moving and relocation      0.998338
purpose_vacation               title_Vacation                   0.997987
id                             member_id                        0.997540
purpose_home_improvement 

In [20]:
#save top correlations to dataframe
corr_df_filter = pd.DataFrame(get_top_abs_correlations(y2015, 150))

In [21]:
corr_df_filter.head()

,,0
loan_amnt,funded_amnt,1.0
term_ 36 months,term_ 60 months,1.0
initial_list_status_f,initial_list_status_w,1.0
purpose_educational,title_Learning and training,1.0
application_type_INDIVIDUAL,application_type_JOINT,1.0


In [22]:
#corr_df_filter.reset_index()

In [23]:
#make dataframe into 3 columns instead of 1 bc var names were treated as indices
corr_df_filter = corr_df_filter.reset_index()

In [24]:
#add column names that are easy to work with
corr_df_filter.columns = ['var1','var2', 'corr']

In [25]:
corr_df_filter.head()

,var1,var2,corr
0,loan_amnt,funded_amnt,1.0
1,term_ 36 months,term_ 60 months,1.0
2,initial_list_status_f,initial_list_status_w,1.0
3,purpose_educational,title_Learning and training,1.0
4,application_type_INDIVIDUAL,application_type_JOINT,1.0


In [26]:
corr_df_filter[corr_df_filter['var1'] == 'loan_status_Current']

,var1,var2,corr
37,loan_status_Current,last_pymnt_d_Jan-2017,0.822902
48,loan_status_Current,loan_status_Fully Paid,0.753602
60,loan_status_Current,next_pymnt_d_Feb-2017,0.726260
99,loan_status_Current,last_credit_pull_d_Jan-2017,0.619326


In [27]:
#see variables highly correlated with your target
target_correlations = corr_df_filter[corr_df_filter['var1'] == 'loan_status_Current']

In [28]:
print(target_correlations)

                   var1                         var2      corr
37  loan_status_Current        last_pymnt_d_Jan-2017  0.822902
48  loan_status_Current       loan_status_Fully Paid  0.753602
60  loan_status_Current        next_pymnt_d_Feb-2017  0.726260
99  loan_status_Current  last_credit_pull_d_Jan-2017  0.619326


In [29]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status_Current', 1)
Y = y2015['loan_status_Current']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([0.99772031, 0.98705771, 0.99430064, 0.99574923, 0.92797264,
       0.98898098, 0.90598209, 0.99829015, 0.82825524, 0.92182194])